<a href="https://colab.research.google.com/github/vsukamet/NLPProject/blob/master/OOS_BINARY_METHOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import sys
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
df = json.loads(requests.get('https://raw.githubusercontent.com/clinc/oos-eval/master/data/binary_undersample.json').text)
df_full = json.loads(requests.get('https://raw.githubusercontent.com/clinc/oos-eval/master/data/data_oos_plus.json').text)

In [0]:
''' Getting Training data '''
train = pd.DataFrame(df['train'],columns=['query','label'])
train_inscope = pd.DataFrame(df_full['train'],columns=['query','intent'])

''' Getting Validation data '''
val = pd.DataFrame(df['val'],columns=['query','label'])
val_inscope = pd.DataFrame(df_full['val'],columns=['query','intent'])

''' Getting Test data '''
test = pd.DataFrame(df['test'],columns=['query','label'])
test_inscope = pd.DataFrame(df_full['test'],columns=['query','intent'])

In [0]:
inscope_df_full = pd.concat([train_inscope,val_inscope,test_inscope])
inscope_df_full

,query,intent
0,what expression would i use to say i love you ...,translate
1,can you tell me how to say 'i do not speak muc...,translate
2,"what is the equivalent of, 'life is good' in f...",translate
3,"tell me how to say, 'it is a beautiful morning...",translate
4,"if i were mongolian, how would i say that i am...",translate
...,...,...
4495,why can't i use my credit card,card_declined
4496,why won't you let me pay with my credit card,card_declined
4497,why did i get rejected on my card,card_declined
4498,how come my credit card isn't working,card_declined


In [0]:
#First_Stage_Classification (Preprocessing Step)

train = train.append(val).reset_index(drop=True)

X_train = train['query'].values
Y_train = train['label'].values

# X_val = val['query'].values
# Y_val = val['label'].values

X_test = test['query'].values
Y_test = test['label'].values

#Transforming the text labels into numerics
from sklearn import preprocessing

train_enc = preprocessing.LabelEncoder()
train_enc.fit(Y_train)

Y_train = train_enc.transform(Y_train)
# Y_val = train_enc.transform(Y_val)
Y_test = train_enc.transform(Y_test)

label_dict = dict(zip(list(train_enc.classes_), train_enc.transform(list(train_enc.classes_))))
print(label_dict)

{'in': 0, 'oos': 1}


In [0]:
#First_Stage_Classification (Tokenizing the text)
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

# TF-IDF BASED FEATURE REPRESENTATION
tfidf_model=TfidfVectorizer(use_idf=True, max_df=0.95)
tfidf_model.fit_transform(X_train)

X_train=tfidf_model.transform(X_train)
# X_val=tfidf_model.transform(X_val)
X_test=tfidf_model.transform(X_test)
type(X_test)

scipy.sparse.csr.csr_matrix

In [0]:
#First_Stage_Classification (Training the model)

from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC


model = SVC(kernel='linear',C=1)

# model = MLPClassifier(hidden_layer_sizes=(400, ), activation='tanh', solver='adam', alpha=0.0001, batch_size='auto',
#                                    learning_rate='constant', learning_rate_init=0.001, power_t=0.5,
#                                    max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False,
#                                    warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False,
#                                    validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10)

model.fit(X_train,Y_train)

# y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

# val_intent_predictions = train_enc.inverse_transform(y_val_pred)
test_intent_predictions = train_enc.inverse_transform(y_test_pred)

type(test_intent_predictions)


numpy.ndarray

In [0]:
First_Stage_Val_Predictions = np.vstack([test['query'].values,test_intent_predictions])
First_Stage_Val_Predictions = First_Stage_Val_Predictions.T
First_Stage_Val_Pred_inscope = First_Stage_Val_Predictions[First_Stage_Val_Predictions[:,1] == 'in']

In [0]:
#Second-Stage Classification

#Getting the True-labels of In-Scope-Queries for Train Dataframe to Train the Model
inscope_train_queries = train[train.label == 'in']
inscope_train_undersample = pd.merge(inscope_train_queries,inscope_df_full,on='query',how = 'inner')


X_inscope_train = inscope_train_undersample['query'].values
Y_inscope_train = inscope_train_undersample['intent'].values

#Transforming the text labels into numerics
train_enc_inscope = preprocessing.LabelEncoder()
train_enc_inscope.fit(Y_inscope_train)
Y_train_inscope = train_enc_inscope.transform(Y_inscope_train)


# TF-IDF BASED FEATURE REPRESENTATION
tfidf_model_2 = TfidfVectorizer(use_idf=True, max_df=0.95)
tfidf_model_2.fit_transform(X_inscope_train)
X_train_inscope = tfidf_model_2.transform(X_inscope_train)

X_test_inscope = tfidf_model_2.transform(First_Stage_Val_Pred_inscope[:,0])

# #Training the MLP Model again to further Classify

model_2 = SVC(kernel='linear',C=1)
# model_2 = MLPClassifier(hidden_layer_sizes=(400, ), activation='tanh', solver='adam', alpha=0.0001, batch_size='auto',
#                                    learning_rate='constant', learning_rate_init=0.001, power_t=0.5,
#                                    max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False,
#                                    warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False,
#                                    validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10)

model_2.fit(X_train_inscope,Y_train_inscope)

y_test_pred_inscope = model_2.predict(X_test_inscope)

test_predictions_inscope = train_enc_inscope.inverse_transform(y_test_pred_inscope)




In [0]:
First_Stage_Predictions = pd.DataFrame({'query':test['query'].values,'intent':test_intent_predictions})
Second_Stage_inscope_Predictions = pd.DataFrame({'query':First_Stage_Val_Pred_inscope[:,0],'intent':test_predictions_inscope})
# print(First_Stage_Predictions)
# print(Second_Stage_inscope_Predictions)

In [0]:
final_predictions = First_Stage_Predictions.merge(Second_Stage_inscope_Predictions,on='query',how='left')
final_predictions["intent_y"].fillna("oos", inplace = True) 
final_predictions

,query,intent_x,intent_y
0,how would you say fly in italian,in,translate
1,what's the spanish word for pasta,in,definition
2,how would they say butter in zambia,in,translate
3,how do you say fast in spanish,in,translate
4,what's the word for trees in norway,in,travel_suggestion
...,...,...,...
5495,find my wallet,in,w2
5496,can you give me the gps location of harvey,in,current_location
5497,where's my buddy steve right this second,in,current_location
5498,locate jenny at her present position,in,restaurant_reservation


In [0]:
# inscope_test_queries = test[test.label == 'in']
# print(inscope_test_queries.shape)
test_original = pd.merge(test,inscope_df_full,on='query',how = 'left')
test_original = test_original.drop_duplicates(subset=['query'])
test_original["intent"].fillna("oos", inplace = True) 
test_original

,query,label,intent
0,how would you say fly in italian,in,translate
1,what's the spanish word for pasta,in,translate
2,how would they say butter in zambia,in,translate
3,how do you say fast in spanish,in,translate
4,what's the word for trees in norway,in,translate
...,...,...,...
5497,find my wallet,oos,oos
5498,can you give me the gps location of harvey,oos,oos
5499,where's my buddy steve right this second,oos,oos
5500,locate jenny at her present position,oos,oos


In [0]:
# Finding In-Scope Accuracies
from sklearn.metrics import accuracy_score

y_Test = test_original['intent'].values
y_test_pred = final_predictions["intent_y"].values

boolean_test = (y_Test != 'oos')
y_test_inclass = y_Test[y_Test != 'oos']
y_test_pred_inclass = [y_test_pred[i] for i in range(len(y_test_pred)) if boolean_test[i]]

accuracy_score(y_true = y_test_inclass,y_pred = y_test_pred_inclass)

0.8464444444444444

In [0]:
#Finding Out_of_Scope Recall

from collections import Counter

true_positives_test = Counter(np.logical_and(y_test_pred == 'oos',y_Test == 'oos'))[True]
false_negatives_test = Counter(np.logical_and(y_test_pred != 'oos',y_Test == 'oos'))[True]

test_recall = (true_positives_test)/(true_positives_test+false_negatives_test)

test_recall

0.123